## Создание ассистентов

В этом ноутбуке мы попробуем создать и протестировать чат-ассистента на основе Yandex Assistant API, RAG и Function Calling. Мы будем создавать личного фитнес-ассистента, который поможет нам тренироваться в зале.

Для начала, установим Yandex Cloud ML SDK:

In [ ]:
%pip install --upgrade --quiet yandex-cloud-ml-sdk==0.9.1 pydantic==2.11.4

**ВНИМАНИЕ**: После установки библиотек рекомендуется перезапустить Kernel ноутбука.

И ещё пара полезных функций на будущее:

In [2]:
from IPython.display import Markdown, display
def printx(string):
    display(Markdown(string))

Для работы с языковыми моделями нам понадобится ключ `api_key` для сервисного аккаунта, имеющего права на доступ к модели, и `folder_id`. Мы предполагаем, что соответствующие значения хранятся в секретах Datasphere, или установлены в вашей переменной окружения.

Создадим модель последней версии YandexGPT 5 и убедимся, что он кое-что знает про тренировки:

In [3]:
import os
from yandex_cloud_ml_sdk import YCloudML

folder_id = os.environ["folder_id"]
api_key = os.environ["api_key"]

sdk = YCloudML(folder_id=folder_id, auth=api_key)

# Раскомментируйте, если хотите подробнее смотреть, что делает SDK
#sdk.setup_default_logging(log_level='DEBUG')

model = sdk.models.completions("yandexgpt", model_version="rc")

In [4]:
printx(model.run("Как тренироваться, чтобы сбросить вес?").text)

Перед началом любой программы тренировок для снижения веса рекомендуется проконсультироваться с врачом или квалифицированным тренером, чтобы убедиться, что выбранный режим физической активности будет безопасен и эффективен для вашего здоровья и индивидуальных особенностей.

Вот несколько общих рекомендаций по тренировкам для снижения веса:

1. **Комбинируйте кардиотренировки и силовые упражнения:** кардиотренировки помогают сжигать калории и улучшать сердечно-сосудистую систему, а силовые упражнения способствуют укреплению мышц и ускорению метаболизма.

2. **Начните с умеренной интенсивности:** если вы новичок в тренировках, начните с умеренной интенсивности и постепенно увеличивайте нагрузку. Это поможет избежать травм и перетренированности.

3. **Разнообразьте тренировки:** не стоит ограничиваться одним видом тренировок. Разнообразие поможет поддерживать интерес к занятиям и предотвратит привыкание организма к однообразной нагрузке.

4. **Следите за временем и частотой тренировок:** для достижения результатов важно тренироваться регулярно. Рекомендуется заниматься не менее 3–4 раз в неделю по 30–60 минут.

5. **Не забывайте о разминке и заминке:** разминка перед тренировкой помогает подготовить организм к нагрузке, а заминка после тренировки способствует восстановлению.

6. **Контролируйте питание:** физическая активность должна сопровождаться правильным питанием. Следите за балансом калорий и питательными веществами, чтобы создать дефицит калорий для снижения веса.

7. **Будьте терпеливы:** снижение веса — это процесс, который требует времени и усилий. Не ожидайте быстрых результатов и не сдавайтесь, если прогресс кажется медленным.

8. **Записывайте свои достижения:** ведите дневник тренировок или используйте приложения для отслеживания прогресса. Это поможет вам видеть результаты и мотивировать на дальнейшие достижения.

9. **Обратите внимание на своё самочувствие:** если во время тренировок вы чувствуете боль или дискомфорт, прекратите занятия и обратитесь к врачу.

10. **Достаточный отдых:** важно давать организму время на восстановление между тренировками. Отдых помогает предотвратить перетренированность и травмы.

## Assistant API

Для ведения беседы с моделью с сохранением контекста диалога используем Assistants API. Объект `thread` будет отвечать за сохранение контекста, а `assistant` - за все основные установки, связанные с работой ассистента.

In [5]:
def create_thread():
    return sdk.threads.create(ttl_days=1, expiration_policy="static")

def create_assistant(model, tools=None):
    kwargs = {}
    if tools and len(tools) > 0:
        kwargs = {"tools": tools}
    return sdk.assistants.create(
        model, ttl_days=1, expiration_policy="since_last_active", **kwargs
    )

Создадим простого ассистента и беседу:

In [ ]:
thread = create_thread()
assistant = create_assistant(model)

assistant.update(
    instruction="Ты - опытный фитнес-тренер, задача которого - помочь мне тренироваться в зале."
)

thread.write("Привет! С чего ты порекомендуешь начать тренировки в зале?")

run = assistant.run(thread)
result = run.wait()

printx(result.text)

Привет! Начать тренировки в зале следует с нескольких важных шагов:

1. **Консультация с врачом.** Прежде чем приступить к тренировкам, убедитесь, что ваше здоровье позволяет заниматься спортом. Это особенно важно, если у вас есть хронические заболевания или вы давно не занимались физической активностью.

2. **Определение целей.** Подумайте, чего вы хотите достичь: похудеть, набрать мышечную массу, улучшить общую физическую форму или что-то другое. Это поможет разработать эффективную программу тренировок.

3. **Разработка программы тренировок.** Лучше всего обратиться к профессиональному тренеру, который поможет составить индивидуальный план, учитывая ваши цели, физическую подготовку и особенности здоровья. Если вы предпочитаете самостоятельные тренировки, начните с базовых упражнений, таких как приседания, отжимания, подтягивания и упражнения на пресс.

4. **Правильная техника выполнения упражнений.** Освойте правильную технику выполнения каждого упражнения, чтобы избежать травм и достичь лучших результатов. Если вы новичок, лучше начать с работы под руководством тренера или внимательно изучить технику по видеоурокам и инструкциям.

5. **Разминка.** Перед каждой тренировкой проводите разминку, чтобы подготовить мышцы и суставы к нагрузке. Разминка может включать лёгкий кардио (бег на месте, прыжки со скакалкой) и динамические упражнения на растяжку.

6. **Постепенное увеличение нагрузки.** Не стремитесь сразу выполнить большое количество повторений или поднять максимальный вес. Увеличивайте нагрузку постепенно, давая организму время адаптироваться.

7. **Отдых и восстановление.** Дайте своему организму время на восстановление между тренировками. Включите в свой график дни отдыха или занимайтесь разными видами активности в разные дни, чтобы избежать переутомления мышц.

8. **Правильное питание.** Уделяйте внимание своему рациону, так как питание играет важную роль в достижении спортивных целей. Убедитесь, что вы получаете достаточное количество белков, углеводов и жиров, а также витаминов и минералов.

9. **Гидратация.** Не забывайте пить достаточное количество воды во время тренировок и в течение дня.

10. **Мониторинг прогресса.** Ведите дневник тренировок, записывая упражнения, веса, повторения и свои ощущения. Это поможет вам отслеживать прогресс и вносить необходимые корректировки в программу.

In [7]:
thread.write("Я хочу похудеть!")

run = assistant.run(thread)
result = run.wait()

printx(result.text)

Отлично! Вот несколько рекомендаций, которые помогут вам похудеть при тренировках в зале:

1. **Комбинируйте кардио и силовые тренировки.** Кардиоупражнения (бег, езда на велосипеде, эллипс) помогут сжечь калории, а силовые тренировки (работа с гантелями, штангой, собственным весом) помогут укрепить мышцы и ускорить метаболизм.

2. **Соблюдайте режим тренировок.** Чтобы похудеть, важно заниматься регулярно. Начните с 2–3 тренировок в неделю и постепенно увеличивайте их количество и интенсивность.

3. **Контролируйте калорийность питания.** Для похудения необходимо создать дефицит калорий, то есть тратить больше калорий, чем вы потребляете. Однако не снижайте калорийность рациона слишком резко, чтобы не навредить здоровью.

4. **Уделяйте внимание белку в рационе.** Белок помогает сохранить мышечную массу при похудении и ускоряет метаболизм. Включайте в свой рацион источники белка, такие как мясо, рыба, яйца, молочные продукты, бобовые.

5. **Не забывайте о восстановлении.** Дайте своему организму время на восстановление между тренировками. Спите достаточное количество времени и следите за общим состоянием здоровья.

6. **Будьте терпеливы.** Похудение — это процесс, который требует времени и усилий. Не ожидайте быстрых результатов и не сдавайтесь, если прогресс кажется медленным.

7. **Консультация с тренером и диетологом.** Если вы не уверены, как правильно составить программу тренировок и питания для похудения, обратитесь за помощью к профессионалам. Они помогут вам разработать индивидуальный план, учитывая ваши цели, физическую подготовку и особенности здоровья.

В итоге в переписке `thread` содержится вся история сообщений:

In [8]:
for msg in list(thread)[::-1]:
    printx(f"**{msg.author.role}:** {msg.text}")

**USER:** Привет! С чего ты порекомендуешь начать тренировки в зале?

**ASSISTANT:** Привет! Начать тренировки в зале следует с нескольких важных шагов:

1. **Консультация с врачом.** Прежде чем приступить к тренировкам, убедитесь, что ваше здоровье позволяет заниматься спортом. Это особенно важно, если у вас есть хронические заболевания или вы давно не занимались физической активностью.

2. **Определение целей.** Подумайте, чего вы хотите достичь: похудеть, набрать мышечную массу, улучшить общую физическую форму или что-то другое. Это поможет разработать эффективную программу тренировок.

3. **Разработка программы тренировок.** Лучше всего обратиться к профессиональному тренеру, который поможет составить индивидуальный план, учитывая ваши цели, физическую подготовку и особенности здоровья. Если вы предпочитаете самостоятельные тренировки, начните с базовых упражнений, таких как приседания, отжимания, подтягивания и упражнения на пресс.

4. **Правильная техника выполнения упражнений.** Освойте правильную технику выполнения каждого упражнения, чтобы избежать травм и достичь лучших результатов. Если вы новичок, лучше начать с работы под руководством тренера или внимательно изучить технику по видеоурокам и инструкциям.

5. **Разминка.** Перед каждой тренировкой проводите разминку, чтобы подготовить мышцы и суставы к нагрузке. Разминка может включать лёгкий кардио (бег на месте, прыжки со скакалкой) и динамические упражнения на растяжку.

6. **Постепенное увеличение нагрузки.** Не стремитесь сразу выполнить большое количество повторений или поднять максимальный вес. Увеличивайте нагрузку постепенно, давая организму время адаптироваться.

7. **Отдых и восстановление.** Дайте своему организму время на восстановление между тренировками. Включите в свой график дни отдыха или занимайтесь разными видами активности в разные дни, чтобы избежать переутомления мышц.

8. **Правильное питание.** Уделяйте внимание своему рациону, так как питание играет важную роль в достижении спортивных целей. Убедитесь, что вы получаете достаточное количество белков, углеводов и жиров, а также витаминов и минералов.

9. **Гидратация.** Не забывайте пить достаточное количество воды во время тренировок и в течение дня.

10. **Мониторинг прогресса.** Ведите дневник тренировок, записывая упражнения, веса, повторения и свои ощущения. Это поможет вам отслеживать прогресс и вносить необходимые корректировки в программу.

**USER:** Я хочу похудеть!

**ASSISTANT:** Отлично! Вот несколько рекомендаций, которые помогут вам похудеть при тренировках в зале:

1. **Комбинируйте кардио и силовые тренировки.** Кардиоупражнения (бег, езда на велосипеде, эллипс) помогут сжечь калории, а силовые тренировки (работа с гантелями, штангой, собственным весом) помогут укрепить мышцы и ускорить метаболизм.

2. **Соблюдайте режим тренировок.** Чтобы похудеть, важно заниматься регулярно. Начните с 2–3 тренировок в неделю и постепенно увеличивайте их количество и интенсивность.

3. **Контролируйте калорийность питания.** Для похудения необходимо создать дефицит калорий, то есть тратить больше калорий, чем вы потребляете. Однако не снижайте калорийность рациона слишком резко, чтобы не навредить здоровью.

4. **Уделяйте внимание белку в рационе.** Белок помогает сохранить мышечную массу при похудении и ускоряет метаболизм. Включайте в свой рацион источники белка, такие как мясо, рыба, яйца, молочные продукты, бобовые.

5. **Не забывайте о восстановлении.** Дайте своему организму время на восстановление между тренировками. Спите достаточное количество времени и следите за общим состоянием здоровья.

6. **Будьте терпеливы.** Похудение — это процесс, который требует времени и усилий. Не ожидайте быстрых результатов и не сдавайтесь, если прогресс кажется медленным.

7. **Консультация с тренером и диетологом.** Если вы не уверены, как правильно составить программу тренировок и питания для похудения, обратитесь за помощью к профессионалам. Они помогут вам разработать индивидуальный план, учитывая ваши цели, физическую подготовку и особенности здоровья.

После использования переписку и ассистента можно удалить.

In [9]:
thread.delete()
assistant.delete()

## Добавляем Function Calling

Наш агент может вести переписку и давать советы по фитнесу. Добавим к нему полезную функцию - ведения дневника упражнений. Для этого агенту нужно дать возможность писать некоторую информацию в специальную базу данных. Для простоты будем использовать просто список объектов в памяти, хотя на практике, конечно, имеет смысл использвать какую-нибудь СУБД.

Информация о каждом упражнении будем хранить в виде такого объекта:

In [ ]:
from pydantic import BaseModel, Field
from typing import Optional
from datetime import datetime

class Exercise(BaseModel):
    """Эта функция позволяет добавлять информацию о сделанном в зале упражнении."""

    тип: str = Field(description="Тип упражнения (кардио или силовое)", default=None)
    название: str = Field(description="Название упражения", default=None)
    болевые_ощущения : str = Field(description="Болевые ощущения при выполнении упражнения", default=None)
    пульс : int = Field(description="Пульс в момент выполнения упражнения", default=None)
    подходы : int = Field(description="Количество подходов", default=None)
    повторения : int = Field(description="Количество повторений", default=None)

Добавим к нашему боту возможность вызова функции. Для этого при создании ассистента в списке `tools` укажем соответствующий инструмент на основе описанного нами класса - это приведёт к тому, что каждый раз при вызове модели ей будет передаваться подсказка, что ей доступна функция с указанным описанием. 

> **ВАЖНО**: Описание семантики функции берётся из doc-строки в классе `Exercise`, а описания параметров функции - из соответствующих полей `description` каждого поля. Поэтому в классе `Exercise` очень важно подробно документировать все поля.

Также, для надёжности, мы опишем критерии вызова функции в системном промпте.

In [11]:
add_exercise_tool = sdk.tools.function(Exercise)

assistant = create_assistant(model, tools=[add_exercise_tool])
thread = create_thread()

instruction = """
Ты - опытный фитнес-тренер, задача которого - помочь мне тренироваться в зале. Ты можешь
советовать упражнения, давать рекомендации по питанию и т.д. Ты также можешь вести 
дневник выполненных пользователем упражнений - для этого используй функцию `Exercise`.
"""

_ = assistant.update(instruction=instruction)

In [12]:
thread.write("Привет! Я сделал 10 приседаний с отягощением.")
run = assistant.run(thread)
res = run.wait()
res

RunResult(status=<RunStatus.TOOL_CALLS: 5>, error=None, tool_calls=ToolCallList(ToolCall(function=FunctionCall(name='Exercise', arguments={'подходы': 1.0, 'тип': 'силовое', 'повторения': 10.0, 'название': 'Приседания с отягощением'})),), _message=None, usage=Usage(input_text_tokens=284, completion_tokens=33, total_tokens=317))

Видим, что модель хочеть совершить вызов функции, поэтому возвращает результат со статусом `TOOL_CALLS`, который содержит в себе имя функции и всё необходимые аргументы. Реализовать сам вызов функции - это наша задача:

In [13]:
exercise_db = {}

def add_exercise(thread, exercise):
    if thread.id not in exercise_db:
        exercise_db[thread.id] = []
    exercise_db[thread.id].append(exercise)

In [19]:
import time

thread = create_thread()

thread.write("Привет! Я сделал 10 приседаний с отягощением.")

run = assistant.run(thread)
res = run.wait()
if res.tool_calls:
    result = []
    for f in res.tool_calls:
        print(f" + Вызываю функцию {f.function.name}")
        x = Exercise.model_validate(f.function.arguments)
        x = add_exercise(thread,x)
        result.append({"name": f.function.name, "content": x})
    run.submit_tool_results(result)
    res = run.wait()

res

 + Вызываю функцию Exercise


RunResult(status=<RunStatus.COMPLETED: 4>, error=None, tool_calls=None, _message=Message(id='fvtm7eh4smts60o7n6di', parts=('Отлично, приседания с отягощением добавлены в твой дневник упражнений. Продолжай в том же духе! Если у тебя есть еще какие-то упражнения или вопросы, не стесняйся спрашивать.',), thread_id='fvta4ule7gr66bvng4s8', created_by='ajej20rll4tifkelclga', created_at=datetime.datetime(2025, 7, 31, 8, 28, 18, 363237), labels=None, author=Author(id='fvtamgj0glttf571q5rj', role='ASSISTANT'), citations=(), status=<MessageStatus.COMPLETED: 1>), usage=Usage(input_text_tokens=609, completion_tokens=67, total_tokens=676))

Таким образом, для вызова функции необходимо:
* Сообщить модели о доступных функциях
* При вызове модели обработать функциональный вызов, если в результате вызова модель вернула результат со статусом `TOOL_CALLS`

Для реализации полноценного ассистента нам потребуется ещё добавить функцию распечатки дневника занятий. Поэтому немного структурируем наш код:
* Добавим функцию для обработки вызова прямо в класс с описанием данных
* Создадим класс `Assistant`, который будет реализовывать функциональный вызов

In [20]:
class Exercise(BaseModel):
    """Эта функция позволяет добавлять информацию о сделанном в зале упражнении."""

    тип: str = Field(description="Тип упражнения (кардио или силовое)", default=None)
    название: str = Field(description="Название упражения", default=None)
    болевые_ощущения : str = Field(description="Болевые ощущения при выполнении упражнения", default=None)
    пульс : int = Field(description="Пульс в момент выполнения упражнения", default=None)
    подходы : int = Field(description="Количество подходов", default=None)
    повторения : int = Field(description="Количество повторений", default=None)

    def process(self, thread):
        if thread.id not in exercise_db:
            exercise_db[thread.id] = []
        exercise_db[thread.id].append(self)

class ListExercises(BaseModel):
    """Эта функция позволяет получить список сделанных упражнений"""

    def process(self,thread):
        if thread.id not in exercise_db:
            return "Упражнений нет"
        else:
            return '\n'.join([
                f"{i+1}. {x.название} ({x.тип})" for i,x in enumerate(exercise_db[thread.id])
            ])


In [31]:
class Assistant():
    def __init__(self, tools, instruction, search_tool = None):
        self.tool_map = { x.__name__ : x for x in tools }
        tools = [ sdk.tools.function(x) for x in tools ]
        if search_tool:
            tools.append(search_tool)
        self.assistant = create_assistant(model,tools)
        self.assistant.update(instruction=instruction)

    def __call__(self, thread):
        run = self.assistant.run(thread)
        res = run.wait()
        if res.tool_calls:
            result = []
            for f in res.tool_calls:
                print(f" + Вызываю функцию {f.function.name}, args = {f.function.arguments}")
                fn = self.tool_map[f.function.name]
                obj = fn(**f.function.arguments)
                x = obj.process(thread)
                result.append({"name": f.function.name, "content": x})
            run.submit_tool_results(result)
            res = run.wait()
        return res

In [28]:
instruction = """
Ты - опытный фитнес-тренер, задача которого - помочь мне тренироваться в зале. Ты можешь
советовать упражнения, давать рекомендации по питанию и т.д. Ты также можешь вести 
дневник выполненных пользователем упражнений - для этого используй функцию `Exercise`. Чтобы
показать список выполненных упражнений - используй `ListExercises`.
"""

assistant = Assistant([Exercise, ListExercises], instruction)

thread = create_thread()
thread.write("Привет! Я сделал 10 отжиманий от пола!")
print(assistant(thread).text)

 + Вызываю функцию Exercise, args = {'тип': 'силовое', 'повторения': 10.0, 'название': 'Отжимания от пола'}
Отлично, 10 отжиманий от пола — это отличное начало! Если у тебя есть еще какие-либо упражнения или вопросы, не стесняйся спрашивать!


In [29]:
thread.write('А какие упражнения я сегодня делал?')
print(assistant(thread).text)

 + Вызываю функцию ListExercises, args = {}
Сегодня ты уже выполнил одно упражнение — 10 отжиманий от пола. Если хочешь, я могу предложить тебе другие упражнения или ответить на твои вопросы!


In [30]:
exercise_db

{'fvt0bcaahrnmsco617gu': [Exercise(тип='силовое', название='Приседания с отягощением', болевые_ощущения=None, пульс=None, подходы=1, повторения=10)],
 'fvta4ule7gr66bvng4s8': [Exercise(тип='силовое', название='Приседания с отягощением', болевые_ощущения=None, пульс=None, подходы=None, повторения=10)],
 'fvtrnbhdeoc14podc437': [Exercise(тип='силовое', название='Отжимания от пола', болевые_ощущения=None, пульс=None, подходы=None, повторения=10)],
 'fvt7585qkqrspr92jqms': [Exercise(тип='силовое', название='Отжимания от пола', болевые_ощущения=None, пульс=None, подходы=None, повторения=10)],
 'fvtf8goha9orm0v0l6va': [Exercise(тип='силовое', название='Отжимания от пола', болевые_ощущения=None, пульс=None, подходы=None, повторения=10)]}

## Добавляем RAG

Наш ассистент сам по себе неплохо отвечает на вопросы по тренировкам, но мы можем сделать его ещё умнее, добавив в него дополнительную текстовую информацию. Для этого мы можем использовать RAG - подход, который позволяет искать релевантную информацию по запросу в текстовых базах знаний.

В качестве текстовой базы знаний возьмём несколько популярных текстов о фитнесе:

In [36]:
from glob import glob
import pandas as pd

def get_token_count(filename):
    with open(filename, "r", encoding="utf8") as f:
        return len(model.tokenize(f.read()))

def get_file_len(filename):
    with open(filename, encoding="utf-8") as f:
        l = len(f.read())
    return l

d = [
    {
        "File": fn,
        "Tokens": get_token_count(fn),
        "Chars": get_file_len(fn),
    }
    for fn in glob("data/text-kb/*.txt")
]

df = pd.DataFrame(d)
df

,File,Tokens,Chars
0,data/text-kb\diary.txt,206,931
1,data/text-kb\how-to-begin.txt,663,3248
2,data/text-kb\program.txt,208,937
3,data/text-kb\qna.txt,1416,7098
4,data/text-kb\what-to-take.txt,205,954


В случае с RAG текстовая база знаний делится на небольшие фрагменты, по которым в ходе запроса осуществляется поиск. Оптимальная длина фрагмента определяется опытным путём, но в среднем разумно придерживаться размера 1000-2000 токенов на фрагмент. В нашем случае длина некоторых файлов превышает 1000 токенов, поэтому мы будем использовать **стратегию чанкования**, для разбиения текстовых файлов на более мелкие фрагементы.

## Загружаем файлы в облако

Чтобы RAG мог осущетвлять поиск по фрагментам файлов, нам необходимо построить индекс, а перед этим - загрузить все файлы в облако.

In [37]:
def upload_file(filename):
    return sdk.files.upload(filename, ttl_days=1, expiration_policy="static")

df["Uploaded"] = df["File"].apply(upload_file)

## Строим индекс

Для индексации файлов можно применять следующие стратегии:
* Поиск по эмбеддингам (векторный поиск) - по всем фрагментам текста вычисляются эмбеддинги, и в процессе поиска осуществляется векторный поиск между эмбеддингом запроса и эмбеддингом фрагмента. Это позволяет осуществлять семантический поиск
* Поиск по ключевым словам
* Гибридный поиск, в котором различными способами объединяются результаты поиска по ключевым словам и векторного поиска.


In [38]:
from yandex_cloud_ml_sdk.search_indexes import (
    StaticIndexChunkingStrategy,
    HybridSearchIndexType,
    ReciprocalRankFusionIndexCombinationStrategy,
)

op = sdk.search_indexes.create_deferred(
    df["Uploaded"],
    index_type=HybridSearchIndexType(
        chunking_strategy=StaticIndexChunkingStrategy(
            max_chunk_size_tokens=1000, chunk_overlap_tokens=100
        ),
        combination_strategy=ReciprocalRankFusionIndexCombinationStrategy(),
    ),
)
index = op.wait()

## Собираем RAG-ассистента

Выше при описании класса `Assistant` мы уже предусмотрели возможность передать дополнительно **инструмент поиска**, поэтому для добавления RAG нам достаточно определить такой инструмент поверх индекса и указать его при создании ассистента. Также важно задать хорошую инструкцию (системный промпт): 

In [41]:
search_tool = sdk.tools.search_index(index)

instruction = """
Ты - опытный фитнес-тренер, задача которого - помочь мне тренироваться в зале. Ты можешь
советовать упражнения, давать рекомендации по питанию и т.д. Отвечай на основе имеющейся
дополнительной информации. Ты также можешь вести 
дневник выполненных пользователем упражнений - для этого используй функцию `Exercise`. Чтобы
показать список выполненных упражнений - используй `ListExercises`.
"""

assistant = Assistant([Exercise, ListExercises], instruction, search_tool=search_tool)

thread = create_thread()
thread.write("Что нужно, чтобы начать заниматься в зале?")

res = assistant(thread)
printx(res.text)

Чтобы начать заниматься в зале, вам необходимо:

1. Оценить состояние своего здоровья и поставить цель тренировок (похудение, набор мышечной массы, поддержание формы и т. д.).
2. Выбрать удобную одежду и обувь, которые не сковывают движения.
3. Рассмотреть возможность приобретения пульсометра для контроля пульса во время тренировок.
4. Записаться на вводный инструктаж с тренером, который объяснит работу тренажёров и правильную технику выполнения упражнений.
5. Составить тренировочный план, учитывая свои цели и график.
6. Не забывать о важности разминки перед тренировкой и заминки после неё.
7. Быть готовым постепенно увеличивать нагрузку и давать организму время на восстановление.

Посмотрим, из каких источников был получен этот ответ:

In [43]:
def print_citations(result):
    for citation in result.citations:
        for source in citation.sources:
            if source.type != "filechunk":
                continue
            print("------------------------")
            print(source.parts[0])

print_citations(res)

------------------------
### Если я хочу начать заниматься в спортзале, то с чего мне стоит начать? В первую очередь нужно оценить состояние своего здоровья и поставить цель. От этого будет зависеть, в каком направлении вы будете работать: силовые тренировки, пилатес, кроссфит, функциональный тренинг или что-то другое. Выберите удобную одежду, которая не сковывает движения, и кроссовки с амортизацией. Также стоит приобрести пульсометр, чтобы контролировать свой ритм и не перегружать сердце во время упражнений. Для каждого возраста существует свой максимальный диапазон частоты сердечных сокращений. Его можно рассчитать по формуле: (220- свой возраст) х 70%/80%/90%. Максимально допустимая частота для безопасного тренинга – 90%. ### Что мне делать в первый день в зале? При первом посещении вы должны записаться на вводный инструктаж, где тренер объяснит работу тренажеров и правильную технику выполнения упражнений. Он входит в стоимость абонемента. Как правило, в первый день вы только изуча

## Добавляем таблицу добавок

Для серьезных тренировок важны также пищевые добавки. Чтобы добавить информацию о них в нашего ассистента, мы нашли таблицу таких добавок в формате markdown:

In [44]:
with open("data/additives.md", encoding="utf-8") as f:
    additives = f.readlines()
additives_all = "".join(additives)

tokens = len(model.tokenize(additives_all))
print(f"Токенов: {tokens}, {len(additives_all)/tokens} chars/token")

Токенов: 6454, 5.105361016423923 chars/token


In [52]:
printx(additives_all[:1040])

| Название добавки                                 | Категория                                      | Назначение                                                                                                 | Доза                          | Рейтинг |
|--------------------------------------------------|-----------------------------------------------|-----------------------------------------------------------------------------------------------------------|-------------------------------|---------|
| Креатин                                          | Для наращивания мышечной массы, силы и ускорения восстановления | Увеличивает физическую силу, ускоряет восстановление; увеличивает массу                                   | 2-20 г в день                 | *****   |
| Глютамин                                         | Для наращивания мышечной массы, силы и ускорения восстановления | Предотвращает распад мышечной ткани; укрепляет иммунную систему                                           | 5-20 г в день                 | *****   

Видим, что табличка большая, поэтому её придётся *чанковать*. Но при этом важно чанковать табличку так, чтобы в каждом фрагмента оставался заголовок таблицы, который определяет семантику столбцов.

Отделим заголовок таблицы:

In [53]:
header = additives[:2]
header

['| Название добавки                                 | Категория                                      | Назначение                                                                                                 | Доза                          | Рейтинг |\n',
 '|--------------------------------------------------|-----------------------------------------------|-----------------------------------------------------------------------------------------------------------|-------------------------------|---------|\n']

Ниже будем чанковать табличку вручную, задав размер чанка в символах для простоты. Мы будем сразу загружать получившиеся фрагменты в облако, минуя диск:

In [54]:
chunk_size = 600 * 3  # около 600 tokens * 5 char/token

s = header.copy()
uploaded_additives = []
for x in additives[2:]:
    s.append(x)
    if len("".join(s)) > chunk_size:
        id = sdk.files.upload_bytes(
            "".join(s).encode(), ttl_days=5, expiration_policy="static",
            mime_type="text/markdown",
        )
        #printx("".join(s))
        uploaded_additives.append(id)
        s = header.copy()
print(f"Uploaded {len(uploaded_additives)} table chunks")

Uploaded 24 table chunks


Теперь добавим эти фрагменты в индекс:

In [56]:
op = index.add_files_deferred(uploaded_additives)
xfiles = op.wait()

Посмотрим, как система стала отвечать на вопросы о добавках:

In [59]:
thread = create_thread()

thread.write("Какие добавки помогают нарастить мышечную массу?")
result = assistant(thread)

printx(result.text)
print_citations(result)

Для наращивания мышечной массы могут помочь следующие добавки:

1. Креатин — увеличивает физическую силу, ускоряет восстановление и способствует увеличению мышечной массы. Доза: 2–20 г в день.
2. Глютамин — предотвращает распад мышечной ткани и укрепляет иммунную систему. Доза: 5–20 г в день.
3. Аминокислоты с разветвлёнными цепями — предотвращают распад мышечной ткани и притупляют чувство усталости. Доза: 5–10 г в день.
4. ZMA (цинк-монометионин-аспартат, магний-аспартат, В6) — повышает уровень тестостерона и ИГФ-1, увеличивает физическую силу и улучшает сон/восстановление. Доза: 2–3 капсулы в день.
5. В-гидрокси-В-метилбутират (НМВ) — увеличивает мышечную массу тела и предотвращает распад мышечного белка. Доза: 3–5 г в день.

------------------------
| Название добавки                                 | Категория                                      | Назначение                                                                                                 | Доза                          | Рейтинг |
|--------------------------------------------------|-----------------------------------------------|-----------------------------------------------------------------------------------------------------------|-------------------------------|---------|
| Креатин                                          | Для наращивания мышечной массы, силы и ускорения восстановления | Увеличивает физическую силу, ускоряет восстановление; увеличивает массу                                   | 2-20 г в день                 | *****   |
| Глютамин                                         | Для наращивания мышечной массы, силы и ускорения восстановления | Предотвращает распад мышечной ткани; укрепляет иммунную систему                    

## Удаляем лишнее

В заключение удалим созданные ресурсы!

> **ВНИМАНИЕ**: Не выполняйте этот код, если у вас есть другие проекты с ассистентами в облаке! Он удаляет все индексы, ассистентов, файлы и переписки! 

In [60]:
for thread in sdk.threads.list():
    print(f" + deleting thread id={thread.id}",end="")
    try:
        thread.delete()
    except:
        print(" ! Error",end="")
    print()
        
for assistant in sdk.assistants.list():
    print(f" + deleting assistant id={assistant.id}")
    assistant.delete()

 + deleting thread id=fvtg97evjkul2bm5oqm8
 + deleting thread id=fvte2kpoilmf3hv5jrn9
 + deleting thread id=fvt093qg93aumd11fil8
 + deleting thread id=fvt7dstj6i3fqmnaeeo2
 + deleting thread id=fvtdnjb29blm9rkn4l8t
 + deleting thread id=fvtfifkrmemi818uefre
 + deleting thread id=fvtf8goha9orm0v0l6va
 + deleting thread id=fvt7585qkqrspr92jqms
 + deleting thread id=fvtrnbhdeoc14podc437
 + deleting thread id=fvta4ule7gr66bvng4s8
 + deleting thread id=fvt0bcaahrnmsco617gu
 + deleting thread id=fvtapq56ncnsr8o75tvk
 + deleting assistant id=fvtksq30fuphd8c9jo2d
 + deleting assistant id=fvt7hubchdgqee07pilj
 + deleting assistant id=fvt2oc2se9f1m312qh4e
 + deleting assistant id=fvt7g7dro7t5tg8ibcfc
 + deleting assistant id=fvttpmq7nsd67kv9n0ah
 + deleting assistant id=fvtkqtvlgmvjjmhueoeq
 + deleting assistant id=fvtamgj0glttf571q5rj


In [61]:
from tqdm.auto import tqdm

for index in sdk.search_indexes.list():
    print(f" + deleting index id={index.id}")
    index.delete()
    
print(" + Deleting files")
for file in tqdm(sdk.files.list()):
    # print(f" + deleting file id={file.id}")
    file.delete()

 + deleting index id=fvteik6lgsf60nmff49t
 + Deleting files


29it [00:05,  5.07it/s]
